# Analysis of Trajectory using MDtraj

MDTraj is a python library that provides tools to analyze and modify your trajectory file.

Utilize it to study the solvation of the solute in your systems.

**Units**
* **time = dumping frequency of the trajectoy**
* **space = nm**

In [ ]:
import mdtraj as md
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# load trajectory

t = md.load_lammpstrj('dump.lammpstrj', top='topology.pdb')

In [ ]:
# initialize the topology object 

top = t.topology
frames = len(t)

print(t)
print(frames)

In [ ]:
#creating a file with topology

table, bonds = top.to_dataframe()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    f = open("topology.txt", "a")
    f.write(str(table))
    f.close

In [ ]:
# atom selection

H2O = (top.select("resname SPC"))
O = (top.select("resname SPC and element O"))
H = (top.select("resname SPC and element H"))
Na = (top.select("resname Na"))
Cl = (top.select("resname Cl"))

# print(O)


In [ ]:
# select pairs of atoms with
# top.select_pairs

## Radial Distribution Function 

The radial distribution function, $g(r)$ in a system of particles gives a measure of the local structure around a reference particle. It describes the local density variation at radius $r$, relative to a uniform particle distribution ($\rho=N/V$).  

The peaks in the RDF define the coordination shells of the reference species. The RDF is the first and best tool to have a general picture of the solvation environment of a solute moclecule.



In [ ]:
# Calculate the RDF between atoms of interest using
# md.compute_rdf 

In [ ]:
# Plot the calculated radial distribution functions and compare with those from LAMMPS

fig, ax = plt.subplots()

ax.plot()

ax.set(xlabel='$r$ / Å', ylabel='$g(r)$')
fig.savefig("rdf.png",bbox_inches='tight', dpi=300)


## Coordination Number

The coordination number is the number of observed particles found in the first coordination shell of the reference particle.

This can be obtained by integrating the the $g(r)$ in spherical coordinates up to its first minimum:

$$
n(r_{min}) = 4\pi \int_{0}^{r_{min}} g(r) r^{2} dr
$$

In [ ]:
# Calculate the coordination numbers between atoms of interest 
# and compare with the ones obtained by LAMMPS 

## Combined distribution functions

Several distribution functions can be combined to get more information on the local structure of the systems. 

Hydrogen bonding is defined by close vicinity of an hydrogen atom, covalently bonded to a very electronegative atom, with and another covalently bonded atom.

The H of the donor particle is usually found at less than 3 Å of distance to the hydrogen bond acceptor and the angle between the electronegative species of the donor (D), the H atom(HD) and the acceptor(A) should be at ∠ 180°.

Combining the radial angular distribution function of the D--HD--A angle with the radial distribution function of the HD--A bond, allows us to verify the presence of a hydrogen bond.

In [ ]:
def hbond(DH_A,cutoff,t,top):
    """Calculation of combined angular and radial distribution function

    Args:
        DH_A (array): (N,2) array of H donor and acceptor pairs
        cutoff (float): cutoff value for the RDF
        t (MDTraj obj): Trajectory object
        top (MDTraj obj): Topology oblect

    Returns:
        angle_distance (array): (N',2) array of angle and distance values
    """
    
    frames=len(t)
    angle_distance = []
    
    dist_DHA = md.compute_distances(t, DH_A)
    
    for frame in range(frames):
        for i in range(len(DH_A)):
            if dist_DHA[frame,i] > cutoff:
                continue
            for bond in top.bonds:
                if bond[0].index == DH_A[i,0]:
                    O = bond[1].index
                elif bond[1].index == DH_A[i,0]:
                    O = bond[0].index
            D_DH_A = [[O, DH_A[i,0], DH_A[i,1]]]
            angle_DDHA = md.compute_angles(t,D_DH_A)[frame]*57.2958
            angle_distance.append([angle_DDHA[0],dist_DHA[frame,i]])
            
    return(np.array(angle_distance))

In [ ]:
def bin(coords, nbinx, Lx, nbiny, Ly):
    """Binning angle and distance values
    Args:
        coords: (N,2) array of angle and distance values
        nbinx (int): number of bins for angles
        Lx (float): maximum angle value
        nbiny (int): number of bins for distances
        Ly (float): maximum distance value
    Returns:
        angle_distance (array): (nbinx, nbiny) array of binned occurences
    """
    
    bin_data = np.zeros((nbinx, nbiny))

    voxelx = Lx/nbinx
    voxely = Ly/nbiny

    for i in range(len(coords)):
        vi = int(round((coords[i,0]) / voxelx))
        vj = int(round((coords[i,1]) / voxely))
    
       
        if vi < 0 or vi >= nbinx:
            continue
        
        if vj < 0 or vj >= nbiny:
            continue
        bin_data[vi, vj] += 1
    return(bin_data)

In [ ]:
# Using the hbond() function and binning the results with the bin() function 
# calculate the combined angular and radial distribution function of hydrogen
# bonds

#angle_distance = hbond()
#bin_data = bin()


In [ ]:
fig, ax = plt.subplots()

# x = ...
# y = ...

plt.pcolormesh(x, y, bin_data, cmap='YlGnBu', vmin=np.min(bin_data), vmax=np.max(bin_data))
plt.plot([], [], ' ')

ax.set(xlabel='$r$ / Å', ylabel=r'$\theta$ / deg')#,


cb = plt.colorbar()


dir_save = dir
plt.legend(framealpha=0.0)
fig.savefig("hbond.png", dpi=300)
plt.show()

## Dihedral distribution

A dihedral angle is an angle between two planes. In molecular geometry the planes, that are defined by three atoms, must share two of these atoms. the i--j--k--l dihedral angle measures the angle between the planes defined by i--j--k and j--k--l. 

In [ ]:
# calculate the dihedral distribution with
# md.compute_dihedrals
# and plot it in a histogram

In [ ]:
fig, ax = plt.subplots()

plt.hist()

ax.set(xlabel=r'$\theta$ / deg', ylabel='counts')
fig.savefig("dihed.png",bbox_inches='tight', dpi=300)

## Diffusion coefficient

The dynamic properties of the systems were probed by measuring self-diffusion coefficients, of the spicies in the system.

One method to calculate diffusion coefficient is to exploit Einstein relations which allow to derive diffusivities from the mean squared displacement

\begin{equation}
    2tD = \frac{\left< |\pmb{r}(t) - \pmb{r}(0)|^2 \right>}{3}.  
\end{equation}

By plotting the mean squared displacement as a function of time, we can derive the diffusion coefficient from the slope once the diffusive regime is attained.


In [ ]:
# Calculate the MSD of water in the system using the 
# md.rmsd
# function and compare it with the one obtained with LAMMPS


In [ ]:
fig, ax = plt.subplots()
#ax.set_xlim(0, 8)

ax.plot()

ax.set(xlabel='t / ns', ylabel='MSD / Å$^2$')
fig.savefig("msd.png",bbox_inches='tight', dpi=300)